In [ ]:
import pandas as pd
import sys
from pathlib import Path
import plotly.express as px
import os

# Ajoute le dossier parent à sys.path
chemin_dossier_parent = Path(os.getcwd()).parent
sys.path.append(str(chemin_dossier_parent))
from my_data.db_connect import get_session
from my_data.datasets import get_environment_data, get_lichen_data, get_lichen_species_data, get_tree_data, get_observation_data, get_table_data

In [ ]:
session = get_session()

# Récupération des datasets
environment_df = get_environment_data()
lichen_df = get_lichen_data()
lichen_species_df = get_lichen_species_data()
observation_df = get_observation_data()
table_df = get_table_data()
tree_df = get_tree_data()

In [ ]:
# Rename columns ids for easyer merge
lichen_df.rename(columns={'id':'lichen_id'}, inplace=True)
lichen_species_df.rename(columns={'id':'species_id'}, inplace=True)
observation_df.rename(columns={'id':'observation_id'}, inplace=True)
tree_df.rename(columns={'id':'tree_id'}, inplace=True)

In [ ]:
# Color palette (other options here: https://plotly.com/python/discrete-color/)
base_color_palette = px.colors.qualitative.Set2
pastel_color_palette = px.colors.qualitative.Pastel2

In [ ]:
square_columns = ['sq1', 'sq2', 'sq3', 'sq4', 'sq5']
orientations = ['N', 'E', 'S', 'O']

# Create a mapping dictionary for legend items (orientations)
orientations_mapping = {
    "N": "Nord",
    "E": "Est",
    "S": "Sud",
    "O": "Ouest"
}

## Histogram 3

In [ ]:
# Calculate the number of lichens per orientation for each lichen in the table
for orientation in orientations:
    table_df[orientation] = table_df[square_columns].apply(
        lambda row: sum(orientation in sq for sq in row), axis=1
    )

# Sum of N + E + S + O for each lichen
table_df['sum_quadrat'] = table_df[orientations].sum(axis=1) 

table_df

In [ ]:
# Merge tables
merged_table = table_df.merge(right=lichen_df, how='left').merge(right=lichen_species_df, how='left')
merged_table

In [ ]:
obs_id = 501

site_table = merged_table.query("observation_id == @obs_id")
site_table

In [ ]:
site_table_per_lichen = site_table.groupby(by='lichen_id', as_index=False).agg(
    {
        'name': 'first',
         'N': 'sum',
         'O': 'sum',
         'S': 'sum',
         'E': 'sum',
        'sum_quadrat': 'sum'
    }).sort_values(by='sum_quadrat', ascending=True, ignore_index=True)

site_table_per_lichen

In [ ]:
# Create the bar plot
hist3 = px.bar(
    site_table_per_lichen,
    x=orientations,
    y="name",
    orientation="h",
    color_discrete_sequence=base_color_palette
)

# Update layout
hist3.update_layout(
    title_text="Espèces observées sur le site sélectionné",
    title={"x": 0.5, "y": 0.95, "xanchor": "center"},
    margin=dict(l=20, r=20, t=40, b=20), 
    legend_title_text="Orientation",
    template="plotly_white",
    barcornerradius="30%"
)

# Update axes
hist3.update_xaxes(
    title_text="Nombre",
    showgrid=True,
    gridcolor='rgba(200, 200, 200, 0.5)',
    tickfont=dict(size=14)
)
hist3.update_yaxes(
    title_text="",
    tickfont=dict(size=14)  # Adjust tick font size
)

# Customize hover information
hist3.update_traces(hovertemplate="""
    <b>Espèce:</b> %{y}<br>
    <b>Nombre:</b> %{x}<extra></extra>
""")

# Update the legend labels based on the mapping
hist3.for_each_trace(lambda t: t.update(name=orientations_mapping.get(t.name, t.name)))

# Show the plot
hist3.show()

In [ ]:
fig = px.colors.qualitative.swatches()
fig.show()

## Histogram 4

In [ ]:
# Group by species' type and count them
df_grouped = (
    lichen_df
    .groupby("species_id", as_index=False)
    .size()
    .rename(columns={'size': 'count'})
)

# Merge with species names
df_grouped_species = df_grouped.merge(lichen_species_df[['species_id', 'name']], on='species_id', how='left')

# Sort based on occurrences in descending order
df_grouped_species = df_grouped_species.sort_values(by='count', ascending=False).reset_index(drop=True)
df_grouped_species

In [ ]:
# TODO: the user's selection should be interactive -> to modify in the final Dash

user_selection_species_id = 30
user_selection_species_name = lichen_species_df[lichen_species_df["species_id"] == user_selection_species_id]["name"]
print("Species selected by the user:", user_selection_species_name)

# Index in "df_grouped_species" corresponding to the selected species
idx = df_grouped_species[df_grouped_species["species_id"] == user_selection_species_id].index

# Adjust the color of the selected specie
pastel_color = pastel_color_palette[0]
selected_color = base_color_palette[0]

color_hist4 = [pastel_color] * len(df_grouped_species)
color_hist4[int(idx[0])] = selected_color

# Bar plot
hist4 = px.bar(
    df_grouped_species, 
    x="count", 
    y="name",
    orientation="h",
    color="name",
    color_discrete_sequence=color_hist4,
    title="Espèces les plus observées par les observateurs Lichens GO",
)

# Update layout
hist4.update_layout(
    title={"x": 0.5, "y": 0.95, "xanchor": "center"},
    margin=dict(l=20, r=20, t=40, b=20), 
    template="plotly_white",
    barcornerradius="30%"
)

# Remove the legend
hist4.update(layout_showlegend=False)

# Update axes
hist4.update_xaxes(
    title_text="Nombre",
    showgrid=True,
    gridcolor='rgba(200, 200, 200, 0.5)',
)
hist4.update_yaxes(
    title="",
    tickfont=dict(size=10)  # Adjust tick font size
)

# Customize hover information
hist4.update_traces(hovertemplate="""
    <b>Espèce:</b> %{y}<br>
    <b>Nombre:</b> %{x}<extra></extra>
""")

hist4.show()